In [34]:
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from geopy.geocoders import Nominatim
from tqdm import tqdm
from geopy.distance import geodesic
import folium
from folium.plugins import MarkerCluster
import math
import datetime
import geopandas as gpd
import urllib.request
import requests
import json
import openmeteo_requests
import requests_cache
from shapely.geometry import Polygon, Point
from retry_requests import retry
from shapely.wkt import loads

In [35]:
# grid df
# sample df

grids = pd.read_csv("data_bomen/grid_enriched_200.csv", sep=",", encoding="utf-8")
samples = pd.read_csv("data_bomen/negative_samples_200.csv", sep=",", encoding="utf-8")

In [36]:
grids['geometry'] = grids['geometry'].apply(lambda x: loads(x))
grids_gdf = gpd.GeoDataFrame(grids, geometry='geometry')

grids_gdf['centroid'] = grids_gdf['geometry'].centroid
grids_gdf['middle_lat'] = grids_gdf['centroid'].apply(lambda point: point.y)
grids_gdf['middle_lon'] = grids_gdf['centroid'].apply(lambda point: point.x)

negative_samples = samples.merge(grids_gdf[['middle_lat', 'middle_lon']], left_on='grid_id', right_index=True)

negative_samples.rename(columns={'middle_lat': 'LAT', 'middle_lon': 'LON'}, inplace=True)
columns_order = ['Date', 'grid_id', 'LAT', 'LON'] + [col for col in negative_samples.columns if col not in ['Date', 'grid_id', 'LAT', 'LON']]
negative_samples = negative_samples[columns_order]

negative_samples = negative_samples.drop('Unnamed: 0', axis=1, errors='ignore')

negative_samples = negative_samples.sort_index()

/Users/Reimer/opt/anaconda3/envs/fundamentals-data-science/lib/python3.11/site-packages/geopandas/geoseries.py:645: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  result = super().apply(func, convert_dtype=convert_dtype, args=args, **kwargs)
/Users/Reimer/opt/anaconda3/envs/fundamentals-data-science/lib/python3.11/site-packages/geopandas/geoseries.py:645: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  result = super().apply(func, convert_dtype=convert_dtype, args=args, **kwargs)


In [40]:
# Split the DataFrame into 10 smaller DataFrames
num_splits = 10
df_splits = np.array_split(negative_samples, num_splits)
df_splits[8]


/Users/Reimer/opt/anaconda3/envs/fundamentals-data-science/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


,Date,grid_id,LAT,LON,Hour,has_tree,avg_height,avg_diameter,avg_year,Fraxinus,...,Betula,Prunus,Platanus,Malus,Robinia,Crataegus,Ulmus,Carpinus,Overig,Onbekend
1572,2021-09-23,4920,52.317531,5.037673,10,True,3.000,0.150,0.000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
1573,2023-07-16,13081,52.404018,4.898964,15,True,8.143,0.238,1566.981,0.0,...,3.0,1.0,0.0,0.0,1.0,0.0,7.0,19.0,21.0,4.0
1574,2021-05-21,7457,52.344558,4.955038,13,True,10.857,0.000,1955.273,23.0,...,9.0,1.0,0.0,0.0,0.0,0.0,4.0,11.0,18.0,0.0
1575,2021-03-11,6575,52.335549,4.875353,15,True,10.722,0.000,1720.315,0.0,...,1.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,0.0
1576,2022-02-19,10835,52.380594,4.831084,2,True,8.859,0.375,1477.760,0.0,...,0.0,10.0,0.0,0.0,40.0,0.0,20.0,0.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1763,2023-07-05,10477,52.376991,4.783864,13,True,18.759,0.482,1814.769,29.0,...,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,4.0
1764,2022-02-20,9461,52.366180,4.813377,9,True,17.119,0.000,1965.000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,12.0,0.0
1765,2019-06-06,4209,52.310324,4.957989,1,True,11.343,0.384,1944.607,0.0,...,3.0,6.0,8.0,2.0,0.0,26.0,35.0,3.0,33.0,0.0
1766,2023-07-05,7066,52.340955,4.810426,8,True,13.039,0.000,1894.882,0.0,...,4.0,0.0,4.0,0.0,5.0,0.0,15.0,0.0,19.0,0.0


In [39]:
grids

,geometry,has_tree,avg_height,avg_diameter,avg_year,Fraxinus,Salix,Quercus,Alnus,Overig,...,Betula,Prunus,Platanus,Malus,Robinia,Ulmus,Crataegus,Carpinus,Onbekend,grid_id
0,"POLYGON ((4.64663 52.26618, 4.6495812704125425...",False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,"POLYGON ((4.6495812704125425 52.26618, 4.65253...",False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,"POLYGON ((4.652532540825085 52.26618, 4.655483...",False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
3,"POLYGON ((4.655483811237628 52.26618, 4.658435...",False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
4,"POLYGON ((4.65843508165017 52.26618, 4.6613863...",False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19831,"POLYGON ((5.136540888482056 52.47338720720746,...",False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19831
19832,POLYGON ((5.1394921588945985 52.47338720720746...,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19832
19833,"POLYGON ((5.142443429307141 52.47338720720746,...",False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19833
19834,POLYGON ((5.1453946997196836 52.47338720720746...,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19834
